$\Large \textbf{Deriving a lower bound for hidden confounding in obs studies (Part 2)}$ 

In this notebook, we demonstrate an example of our testing procedure using the semi-synthetic experiments, specifically applied to the Hillstrom dataset. This approach follows the methodology discussed in the tutorial for synthetic experiments, but with a focus on a real-world dataset. We subsample from the Hillstrom randomized trial to create an observational dataset that exhibits a known, true confounding strength.

For a comprehensive understanding of these semi-synthetic experiments, please refer to the appendix in our paper. In this example, we use the observational study as the target population. We study two confounders within the Hillstrom dataset: "history," which is a strong confounder highly correlated with the outcome, and "mens," a weaker confounder with a lesser correlation to the outcome. This example highlights how our test can effectively discern between strong and weak confounders.

In [2]:
# Required imports

import numpy as np
import pandas as pd
from scipy.stats import bootstrap
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

import wandb
from test_confounding.ATE.ate_bounds import BootstrapSensitivityAnalysis
from test_confounding.datasets.semi_synthetic.fetch_hillstrom import (
    CAT_COVAR_HILLSTROM,
    NUM_COVAR_HILLSTROM,
    load_fetch_hillstrom_data,
)
from test_confounding.datasets.semi_synthetic.sampling import (
    resample_data_with_confound_func,
    subsample_df,
)

First, we apply the testing procedure to the strong confounder "history". We identify a lower bound $\Gamma_{LB}=4.0$ for the true confounding strength $\Gamma^{*}=5.0$.

In [3]:
seed = 50
confound_func = {"para_form": "adversarial_with_conf", "true_gamma": 5.0} #true conf strenght 5.0

obs_data_pre_conf, rct_data = load_fetch_hillstrom_data(
    conf_var="history",
    support_var="zip_code",
    split_data=True,
    support_feature_values=[0.0, 1.0],
    proportion_full_support=0.8,
    seed=52,
    target_col="spend",
)

obs_data = resample_data_with_confound_func(
    obs_data_pre_conf,
    confound_func_params=confound_func,
    seed=seed,
)

numeric_covariates = NUM_COVAR_HILLSTROM
categorical_covariates = CAT_COVAR_HILLSTROM

# Define transformer for encoding and normalization
transformer = ColumnTransformer(
    [
        (
            "encoder",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            [col for col in categorical_covariates if col != "history"],
        ),
        (
            "normalizer",
            MinMaxScaler(),
            [col for col in numeric_covariates if col != "history"],
        ),
    ]
)

print("RCT num. samples: ", rct_data.shape[0])  # type: ignore
print()

# Transform the features into one-hot encoding
x_obs_raw = obs_data.drop(["Y", "T", "C"], axis=1)
x_rct_raw = rct_data.drop(["Y", "T", "C"], axis=1)
x_obs_encoded = transformer.fit_transform(x_obs_raw)
x_rct_encoded = transformer.transform(x_rct_raw)

t_rct, y_rct, x_rct = (
    np.array(rct_data["T"].values),
    np.array(rct_data["Y"].values),
    x_rct_encoded,
)
t_obs, y_obs, x_obs = (
    np.array(obs_data["T"].values),
    np.array(obs_data["Y"].values),
    x_obs_encoded,
)

==== DATASET CHECKS ====
Num observations =  64000
P(T=1) =  0.66709375
F-statistic(C, Y) =  1.0763267714114924
P-value: =  2.97898074146969e-11
RCT_ATE = 0.596796066727898
Mean(Y) =  1.05090828125

Feature importance for each feature:
C = 0.54428333
recency = 0.18103991
mens = 0.11426223
womens = 0.032999776
zip_code = 0.04475624
newbie = 0.040160246
channel = 0.042498257

Original data num. examples =  56319
Downsampled data num. examples =  20728
RCT num. samples:  7681



In [4]:
n_bootstrap = 50
user_conf = [3.5, 4.0, 4.5, 5.0, 5.5]

n_rct = x_rct.shape[0]
n_obs = x_obs.shape[0]
alpha_trim = 0.001
x_all = np.concatenate((x_rct, x_obs), axis=0)  
s_all = np.concatenate((np.ones(n_rct), np.zeros(n_obs)))

clf = RandomForestClassifier(random_state=seed)
clf.fit(x_all, s_all)
pi_s = clf.predict_proba(x_all)[:, 1]
O_idx = pi_s > alpha_trim

x_obs_trimm, t_obs_trimm, y_obs_trimm = (
    x_obs[O_idx[n_rct:]],  
    t_obs[O_idx[n_rct:]],
    y_obs[O_idx[n_rct:]],
)
_, t_rct_trimm, y_rct_trimm = (
    x_rct[O_idx[:n_rct]],  
    t_rct[O_idx[:n_rct]],
    y_rct[O_idx[:n_rct]],
)

mask = np.logical_and(O_idx, s_all)
rct_to_obs_ratio = s_all[O_idx].sum() / (s_all[O_idx].size - s_all[O_idx].sum())
ys = (
    2
    * (y_rct_trimm * t_rct_trimm - y_rct_trimm * (1 - t_rct_trimm))
    * (1 - pi_s[mask])
    / pi_s[mask]
)
bootstrap_rct = bootstrap(
    (ys,), np.mean, n_resamples=n_bootstrap, axis=0, random_state=seed
)
std_rct = bootstrap_rct.standard_error
var_rct = np.power(std_rct, 2) * (rct_to_obs_ratio**2)
mean_rct = rct_to_obs_ratio * ys.mean()

bootstrap_sa = BootstrapSensitivityAnalysis(
    sa_name="QB",
    inputs=x_obs_trimm,
    treatment=t_obs_trimm,
    outcome=y_obs_trimm,
    gammas=user_conf,
    seed=seed,
)
bounds_dist = bootstrap_sa.bootstrap(num_samples=n_bootstrap, fast_quantile=True)


Quantile functions are now trained for QB. Starting bootstrap.
Elapsed time for 50 bootstrap samples: 88.32 seconds


In [5]:
from test_confounding.test import run_multiple_ate_hypothesis_test

results_dict_ate_strong_conf = run_multiple_ate_hypothesis_test(mean_rct = mean_rct, var_rct = var_rct, bounds_dist = bounds_dist, alpha = 5.0, user_conf = user_conf, verbose = False)

In [6]:
def display_gamma_info(gamma_dict, user_conf): 
    print("Test Results:")
    print("-" * 30)
    
    for gamma in user_conf:
        if gamma in gamma_dict:
            value = gamma_dict[gamma]
            reject = "Yes" if value['reject'] else "No"
            test_statistic = value['test_statistic']
        else:
            # If gamma is not in gamma_dict, assume hypothesis is accepted
            reject = "No"
            test_statistic = "N/A"

        print(f"Γ: {gamma}")
        print(f"  - Reject Null Hypothesis: {reject}")
        print(f"  - Test Statistic: {test_statistic}")
        print("-" * 30)
    
    gamma_effective = gamma_dict.get('gamma_effective', 'Not available')
    print(f"Γ_{{LB}} (Lower Bound): {gamma_effective}")


display_gamma_info(results_dict_ate_strong_conf, user_conf)

Test Results:
------------------------------
Γ: 3.5
  - Reject Null Hypothesis: Yes
  - Test Statistic: -2.257818286425383
------------------------------
Γ: 4.0
  - Reject Null Hypothesis: No
  - Test Statistic: -1.8654897494732143
------------------------------
Γ: 4.5
  - Reject Null Hypothesis: No
  - Test Statistic: N/A
------------------------------
Γ: 5.0
  - Reject Null Hypothesis: No
  - Test Statistic: N/A
------------------------------
Γ: 5.5
  - Reject Null Hypothesis: No
  - Test Statistic: N/A
------------------------------
Γ_{LB} (Lower Bound): 4.0


Next, we test for the weak confounder "mens". We determine a lower bound $\Gamma_{LB}=2.5$ for the true confounding strength $\Gamma^{*}=5.0$, demonstrating that a confounder-outcome correlation that is lower results in reduced power for the test. An intuition for this phenomenon is provided in the appendix of the paper.

In [7]:
obs_data_pre_conf, rct_data = load_fetch_hillstrom_data(
    conf_var="mens",
    support_var="zip_code",
    split_data=True,
    support_feature_values=[0.0, 1.0],
    proportion_full_support=0.8,
    seed=52,
    target_col="spend",
)

obs_data = resample_data_with_confound_func(
    obs_data_pre_conf,
    confound_func_params=confound_func,
    seed=seed,
)

# Define transformer for encoding and normalization
transformer = ColumnTransformer(
    [
        (
            "encoder",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            [col for col in categorical_covariates if col != "mens"],
        ),
        (
            "normalizer",
            MinMaxScaler(),
            [col for col in numeric_covariates if col != "mens"],
        ),
    ]
)

print("RCT num. samples: ", rct_data.shape[0])  # type: ignore
print()

# Transform the features into one-hot encoding
x_obs_raw = obs_data.drop(["Y", "T", "C"], axis=1)
x_rct_raw = rct_data.drop(["Y", "T", "C"], axis=1)
x_obs_encoded = transformer.fit_transform(x_obs_raw)
x_rct_encoded = transformer.transform(x_rct_raw)

t_rct, y_rct, x_rct = (
    np.array(rct_data["T"].values),
    np.array(rct_data["Y"].values),
    x_rct_encoded,
)
t_obs, y_obs, x_obs = (
    np.array(obs_data["T"].values),
    np.array(obs_data["Y"].values),
    x_obs_encoded,
)

==== DATASET CHECKS ====
Num observations =  64000
P(C=1) =  0.55103125
P(T=1) =  0.66709375
Corr(C, Y) =  0.008599413005626647
P-value =  0.02959294801554325
RCT_ATE = 0.596796066727898
Mean(Y) =  1.05090828125

Feature importance for each feature:
C = 0.18518682
recency = 0.14364073
womens = 0.027128274
zip_code = 0.061120052
newbie = 0.035805676
channel = 0.038889784
history = 0.50822866

Original data num. examples =  56319
Downsampled data num. examples =  20678
RCT num. samples:  7681



In [8]:
user_conf = [1.0, 1.5, 2.0, 2.5]

n_rct = x_rct.shape[0]
n_obs = x_obs.shape[0]
alpha_trim = 0.001
x_all = np.concatenate((x_rct, x_obs), axis=0)  
s_all = np.concatenate((np.ones(n_rct), np.zeros(n_obs)))

clf = RandomForestClassifier(random_state=seed)
clf.fit(x_all, s_all)
pi_s = clf.predict_proba(x_all)[:, 1]
O_idx = pi_s > alpha_trim

x_obs_trimm, t_obs_trimm, y_obs_trimm = (
    x_obs[O_idx[n_rct:]],  
    t_obs[O_idx[n_rct:]],
    y_obs[O_idx[n_rct:]],
)
_, t_rct_trimm, y_rct_trimm = (
    x_rct[O_idx[:n_rct]],  
    t_rct[O_idx[:n_rct]],
    y_rct[O_idx[:n_rct]],
)

mask = np.logical_and(O_idx, s_all)
rct_to_obs_ratio = s_all[O_idx].sum() / (s_all[O_idx].size - s_all[O_idx].sum())
ys = (
    2
    * (y_rct_trimm * t_rct_trimm - y_rct_trimm * (1 - t_rct_trimm))
    * (1 - pi_s[mask])
    / pi_s[mask]
)
bootstrap_rct = bootstrap(
    (ys,), np.mean, n_resamples=n_bootstrap, axis=0, random_state=seed
)
std_rct = bootstrap_rct.standard_error
var_rct = np.power(std_rct, 2) * (rct_to_obs_ratio**2)
mean_rct = rct_to_obs_ratio * ys.mean()

bootstrap_sa = BootstrapSensitivityAnalysis(
    sa_name="QB",
    inputs=x_obs_trimm,
    treatment=t_obs_trimm,
    outcome=y_obs_trimm,
    gammas=user_conf,
    seed=seed,
)
bounds_dist = bootstrap_sa.bootstrap(num_samples=n_bootstrap, fast_quantile=True)

Quantile functions are now trained for QB. Starting bootstrap.
Elapsed time for 50 bootstrap samples: 53.74 seconds


In [9]:
results_dict_ate_weak_conf = run_multiple_ate_hypothesis_test(mean_rct = mean_rct, var_rct = var_rct, bounds_dist = bounds_dist, alpha = 5.0, user_conf = user_conf, verbose = False)
display_gamma_info(results_dict_ate_weak_conf, user_conf)

Test Results:
------------------------------
Γ: 1.0
  - Reject Null Hypothesis: Yes
  - Test Statistic: -3.2714988793552737
------------------------------
Γ: 1.5
  - Reject Null Hypothesis: Yes
  - Test Statistic: -2.687749279436396
------------------------------
Γ: 2.0
  - Reject Null Hypothesis: Yes
  - Test Statistic: -2.0301243315018835
------------------------------
Γ: 2.5
  - Reject Null Hypothesis: No
  - Test Statistic: -1.3864528305077364
------------------------------
Γ_{LB} (Lower Bound): 2.5
